In [1]:
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install timm
# !pip install einops

In [ ]:
import os
import gc
import sys
import glob

import torch
import torch.nn as nn
import albumentations as A

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tqdm
import segmentation_models_pytorch as smp
from sklearn.model_selection import StratifiedKFold,KFold

import tifffile as tiff

torch.backends.cudnn.benchmark = True

In [ ]:
name_archive = 'train_dataset_mc.zip'
os.system(f' cp "drive/My Drive/{name_archive}" "{name_archive}" ')
os.system(f' unzip {name_archive} -d train_folder' )

name_archive = 'test_dataset_mc2.zip'
os.system(f' cp "drive/My Drive/{name_archive}" "{name_archive}" ')
os.system(f' unzip {name_archive} -d test_folder' )

0

In [ ]:
import json

In [ ]:
def read_image(path: str) -> np.ndarray:
    image = cv2.imread(str(path), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def parse_polygon(coordinates: dict, image_size: tuple, mode) -> np.ndarray:
    mask = np.zeros(image_size, dtype=np.float32)
    if len(coordinates) == 1:
        points = [np.int32(coordinates)]
        cv2.fillPoly(mask, points, 1)
    else: 
        points = [np.int32([coordinates[0]])] 
        cv2.fillPoly(mask, points, 1) 
        
        if mode == 1:
            for polygon in coordinates[1:]: 
                points = [np.int32([polygon])] 
                cv2.fillPoly(mask, points, 0) 
    return mask

def parse_mask(shape: dict, image_size: tuple, mode) -> np.ndarray:
    """
    Метод для парсинга фигур из geojson файла
    """
    mask = np.zeros(image_size, dtype=np.float32)
    coordinates = shape['coordinates']
    if shape['type'] == 'MultiPolygon':
        for polygon in coordinates:
            mask += parse_polygon(polygon, image_size, mode)
    else:
        mask += parse_polygon(coordinates, image_size, mode)

    return mask

class_ids = {"vessel": 1}

def read_layout(path: str, image_size: tuple, mode) -> np.ndarray:
    """
    Метод для чтения geojson разметки и перевода в numpy маску
    """
    with open(path, 'r', encoding='cp1251') as f:  # some files contain cyrillic letters, thus cp1251
        json_contents = json.load(f)

    num_channels = 1 + max(class_ids.values())
    mask_channels = [np.zeros(image_size, dtype=np.float32) for _ in range(num_channels)]
    mask = np.zeros(image_size, dtype=np.float32)

    if type(json_contents) == dict and json_contents['type'] == 'FeatureCollection':
        features = json_contents['features']
    elif type(json_contents) == list:
        features = json_contents
    else:
        features = [json_contents]

    for shape in features:
        channel_id = class_ids["vessel"]
        mask = parse_mask(shape['geometry'], image_size, mode)
        mask_channels[channel_id] = np.maximum(mask_channels[channel_id], mask)

    mask_channels[0] = 1 - np.max(mask_channels[1:], axis=0)

    return np.stack(mask_channels, axis=-1)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
image_values = []
mask_values_d = []
mask_values_с = []
paths_values = []

for path in tqdm(sorted(os.listdir('train_folder'))):
    if 'png' in path:
        image = read_image('train_folder/' + path)
        if  path.replace( "png", "geojson") not in os.listdir('train_folder'):
            continue

        mask_d = read_layout('train_folder/' + path.replace( "png", "geojson"), image.shape[:2], 0)
        mask_c = read_layout('train_folder/' + path.replace( "png", "geojson"), image.shape[:2], 1)

        image_values += [image]
        mask_values_с += [mask_c]
        mask_values_d += [mask_d]
        paths_values += [path]

  0%|          | 0/1314 [00:00<?, ?it/s]

In [ ]:
skf = KFold(5, random_state=228, shuffle = True)
split_list = []
new_split_list = []
num_i = 0
for train_index, val_index in skf.split(np.arange(len(image_values)), np.arange(len(image_values)) ):
    split_list += [(train_index, val_index)]

In [ ]:
from coat import *
from daformer import *

In [ ]:
class RGB(nn.Module):
    IMAGE_RGB_MEAN = [0.485, 0.456, 0.406]
    # [0.485, 0.456, 0.406]  # [0.5, 0.5, 0.5]
    IMAGE_RGB_STD = [0.229, 0.224, 0.225]
    # [0.229, 0.224, 0.225]  # [0.5, 0.5, 0.5]

    def __init__(self, ):
        super(RGB, self).__init__()
        self.register_buffer('mean', torch.zeros(1, 3, 1, 1))
        self.register_buffer('std', torch.ones(1, 3, 1, 1))
        self.mean.data = torch.FloatTensor(self.IMAGE_RGB_MEAN).view(self.mean.shape)
        self.std.data = torch.FloatTensor(self.IMAGE_RGB_STD).view(self.std.shape)

    def forward(self, x):
        x = (x - self.mean) / self.std
        return x


class Net(nn.Module):

    def __init__(self,
                 encoder=coat_lite_medium,
                 decoder=daformer_conv3x3,
                 encoder_cfg={},
                 decoder_cfg={},
                 ):
        super(Net, self).__init__()
        decoder_dim = decoder_cfg.get('decoder_dim', 320)

        # ----
        self.rgb = RGB()

        self.encoder = encoder

        encoder_dim = self.encoder.embed_dims
        # [64, 128, 320, 512]

        self.decoder = decoder(
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim,
        )
        self.logit = nn.Sequential(
            nn.Conv2d(decoder_dim, 1, kernel_size=1),
        )
        self.aux = nn.ModuleList([
                    nn.Conv2d(decoder_dim, 1, kernel_size=1, padding=0) for i in range(4)
                ])
    def forward(self, batch):
        x = batch
        x = self.rgb(x)

        B, C, H, W = x.shape
        encoder = self.encoder(x)
        # print([f.shape for f in encoder])

        last, decoder = self.decoder(encoder)
        logit = self.logit(last)
        # print(logit.shape)

        output = {}
        # probability_from_logit = torch.sigmoid(logit)
        output['probability'] = logit
        for i in range(4):
            output[f'aux{i}'] = self.aux[i](decoder[i])

        return output   

In [ ]:
import random

In [ ]:
def transformer(p=1.0):
    return A.Compose([
        A.HorizontalFlip(p = 0.5),
        # A.VerticalFlip(p = 0.5),
        # A.RandomRotate90(p = 0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.25, rotate_limit=15, p=0.25, 
                         border_mode=cv2.BORDER_CONSTANT),
        A.ColorJitter(p = 0.5),
        A.GaussNoise(15, p = 0.5),
        # A.CLAHE(2, p =0.5)
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.PiecewiseAffine(p=0.3),
        ], p=0.3),
        A.OneOf([
            A.HueSaturationValue(10,15,10),
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
    ], p=p)

In [ ]:
TRAIN_PATH = 'train_images/'
class HuBMAPDataset(torch.utils.data.Dataset):
    def __init__(self, mask, imgs, imsize = (1440, 1088), tfms=None):
        self.mask = mask
        self.imgs = imgs
        self.image_size = imsize
        self.tfms = tfms
        
    def img2tensor(self, img,dtype:np.dtype=np.float32, mode =0 ):
        if img.ndim==2 : img = np.expand_dims(img,2)
        img = np.transpose(img,(2,0,1)) # C , H , W
        return torch.from_numpy(img.astype(dtype, copy=False))
    
    def __len__(self):
        return len(self.imgs)
    
    def resize(self, img, interp):
        return  cv2.resize(
            img, self.image_size, interpolation=interp)
    # (1632, 1248)
    def __getitem__(self, idx):
        img = self.imgs[idx]
        img = np.dstack([img, img, img])
        mask = self.mask[idx]
        # img = img.astype(np.float32)/255
        if self.tfms is not None:
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        img = img.astype(np.float32)/255
        if len(mask.shape) == 3:
            mask = mask[:, :, 1]
        return self.img2tensor(self.resize(img , cv2.INTER_CUBIC)) , self.img2tensor(self.resize(mask , cv2.INTER_CUBIC))




In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
DEVICE = 'cuda'

In [ ]:
from tqdm.notebook import tqdm
from torch.cuda.amp import autocast, GradScaler

In [ ]:
def eval_nn(model, val_mask, val_dataloader, th, mode = 0):
    with torch.no_grad():
        model.eval()
        outputs_list = []
        tk0 = tqdm(enumerate(val_dataloader), total = len(val_dataloader))
        i = 0
        loss_func = torch.nn.BCEWithLogitsLoss()
        average_loss = 0
        for batch_number,  (data)  in tk0:
            # hui
            img, mask = data
            img = img.to(DEVICE)
            mask = mask.to(DEVICE)

            with torch.cuda.amp.autocast():
                outputs = model(img)['probability']  
                loss = loss_func(ups(outputs), mask)
                outputs =  torch.sigmoid(outputs)
            
            average_loss += loss.cpu().detach().numpy()
            tk0.set_postfix(loss=average_loss / (batch_number + 1), stage="val")
            
            for k in range(outputs.shape[0]):
                sh = val_mask[i].shape
                outputs_list += [torch.nn.functional.interpolate(outputs[k : k + 1], size=tuple(sh[:-1]),
                                                    mode='bilinear',align_corners=False, antialias=True ).cpu().detach().numpy()]
                i += 1
        
        # if mode == 1:
        #     return outputs_list

        list_score = []
        i = 0
        for true, pred in tqdm(zip(val_mask, outputs_list)):
            list_score += [ score(true[:, :, 1], pred[0][0], th) ]
        print(np.mean(list_score))
        if mode == 1:
            return outputs_list

In [ ]:
!wget https://vcl.ucsd.edu/coat/pretrained/coat_lite_medium_384x384_f9129688.pth

--2022-09-29 20:18:51--  https://vcl.ucsd.edu/coat/pretrained/coat_lite_medium_384x384_f9129688.pth
Resolving vcl.ucsd.edu (vcl.ucsd.edu)... 132.239.147.109
Connecting to vcl.ucsd.edu (vcl.ucsd.edu)|132.239.147.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 535303289 (511M)
Saving to: ‘coat_lite_medium_384x384_f9129688.pth’

coat_lite_medium_38 100%[===================>] 510.50M   847KB/s    in 4m 31s  

2022-09-29 20:23:23 (1.89 MB/s) - ‘coat_lite_medium_384x384_f9129688.pth’ saved [535303289/535303289]



In [ ]:
def score(true, pred, th = 0.25):
    pred = (pred > th).astype('int')
    # print(pred_masks)
    sm =( pred * true).sum()
    if sm == 0:
        return sm
    rc = sm / (true.sum() + 1e-9)
    ac = sm / (pred.sum() + 1e-9)
    sc = 2 * (rc * ac) / ( rc + ac )
    return sc

In [ ]:
mask_values_c = [x.clip(0, 1) for x in mask_values_c]

In [ ]:
drop_ids = []

In [2]:
max_len = 1024
batch_size = 1
valid_batch_size = 1
accumulation_steps = 1
epochs = 24
lr = 1.75e-5
clip_grad_norm = 100
params_train = {'batch_size': batch_size, 'shuffle': True, 'drop_last': False, 'num_workers': 4}
params_valid = {'batch_size': valid_batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 4}
ups = nn.Upsample(scale_factor = 4, mode='bilinear', align_corners=False)

for fold in range(5):
    train_mask = [mask_values_c[i] for i in split_list[fold][0] if i not in drop_ids] 
    train_img = [image_values[i] for i in split_list[fold][0] if i not in drop_ids] 

    train_dataset = HuBMAPDataset(train_mask, train_img, imsize = (1632, 1248), tfms = transformer())
    train_dataloader = DataLoader(train_dataset, **params_train)

    val_mask = [mask_values_c[i] for i in split_list[fold][1]]
    val_img = [image_values[i] for i in split_list[fold][1]]

    val_dataset = HuBMAPDataset(val_mask, val_img, imsize = (1632, 1248), tfms = None)
    val_dataloader = DataLoader(val_dataset, **params_valid)

    num_train_steps = int(len(train_dataset) / batch_size  * epochs)

    encoder = coat_lite_medium()
    checkpoint = 'drive/MyDrive/coat_lite_medium_a750cd63.pth'
    # checkpoint = 'coat_lite_medium_384x384_f9129688.pth'
    checkpoint = torch.load(checkpoint, map_location='cuda')
    state_dict = checkpoint['model']
    encoder.load_state_dict(state_dict,strict=False)

    model = Net(encoder = encoder).cuda()


    optimizer = torch.optim.AdamW(model.parameters(), 5e-5)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, len(train_dataloader) * 8, 2, 1e-6)

    loss_func = torch.nn.BCEWithLogitsLoss()
    # torch.nn.CrossEntropyLoss()
    # loss_func = F.binary_cross_entropy_with_logits()
    scaler = GradScaler()

    for epoch in range(epochs):
        # if epoch != 0:
        #     continue
        model.train()
        average_loss = 0
        tk0 = tqdm(enumerate(train_dataloader), total = len(train_dataloader))
        for batch_number,  (data)  in tk0:
            # hui
            optimizer.zero_grad()
            img, mask = data
            img = img.to(DEVICE)
            mask = mask.to(DEVICE)

            with torch.cuda.amp.autocast():
                outputs = model(img)
                outputs_main = ups(outputs['probability'])
                loss = loss_func(outputs_main, mask)
                for i in range(2, 4):
                    tmp_mask = F.interpolate(mask,size=outputs[f'aux{i}'].shape[-2:], mode='nearest')
                    loss += 0.2 * loss_func(outputs[f'aux{i}'], tmp_mask)
                    # loss += criterion_aux_loss(self.aux[i](decoder[i]),batch['mask'])   

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()

            average_loss += loss.cpu().detach().numpy()
            tk0.set_postfix(loss=average_loss / (batch_number + 1),lr = scheduler.get_last_lr()[0], stage="train", epoch = epoch)

        eval_nn(model, val_mask, val_dataloader, 0.425)

        # torch.save({
        #             'state_dict': model.state_dict()
        #             # 'optimizer': optimizer.state_dict()
        #                 }, f'drive/MyDrive/best_{fold}_{epoch}.pt')
    
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
class HuBMAPDatasetTEST(torch.utils.data.Dataset):
    def __init__(self, imgs, imsize = 768, tfms=None):
        # self.mask = mask
        self.imgs = imgs
        self.image_size = imsize
        self.tfms = tfms
        
    def img2tensor(self, img,dtype:np.dtype=np.float32):
        if img.ndim==2 : img = np.expand_dims(img,2)
        img = np.transpose(img,(2,0,1)) # C , H , W
        return torch.from_numpy(img.astype(dtype, copy=False))
    
    def __len__(self):
        return len(self.imgs)
    
    def resize(self, img, interp):
        return  cv2.resize(
            img,  (1632, 1248), interpolation=interp)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.imgs[idx], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sh = img.shape
        # img = img.astype(np.float32)/255
        if self.tfms is not None:
            augmented = self.tfms(image=img)
            img = augmented['image']
        img = img.astype(np.float32)/255
        return self.img2tensor(self.resize(img , cv2.INTER_AREA)), sh

In [ ]:
full_outputs_list = []
for fold in range(5):
    encoder = coat_lite_medium()
    model = Net(encoder = encoder).cuda()
    model_dict = torch.load(f'drive/MyDrive/best_{fold}_23.pt', map_location='cuda')
    model.load_state_dict(model_dict['state_dict'])

    batch_size = 1

    params_valid = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 4}
    ups = nn.Upsample(scale_factor = 4, mode='bilinear', align_corners=False)
    list_test = ['test_folder/eye_test/' + x for x in os.listdir('test_folder/eye_test')]
    test_dataset = HuBMAPDatasetTEST(list_test, tfms = None)
    test_dataloader = DataLoader(test_dataset, **params_valid)

    with torch.no_grad():
        model.eval()
        outputs_list = []
        tk0 = tqdm(enumerate(test_dataloader), total = len(test_dataloader))
        loss_func = torch.nn.BCEWithLogitsLoss()
        average_loss = 0
        for batch_number,  (data)  in tk0:
            img, shs = data
            shs = np.array([shs[0].cpu().detach().numpy(), shs[1].cpu().detach().numpy()]).T
            img = img.to(DEVICE)

            with torch.cuda.amp.autocast():
                outputs = model(img)['probability']  
                outputs =  torch.sigmoid(outputs)

            i = 0
            for k in range(outputs.shape[0]):
                outputs_list += [torch.nn.functional.interpolate(outputs[k : k + 1], size=tuple(shs[i]),
                                                    mode='bilinear',align_corners=False, antialias=True ).cpu().detach().numpy()]
                i += 1

    full_outputs_list += [outputs_list]

In [ ]:
os.makedirs('ss')
for i, f in tqdm(enumerate(list_test)):

    pred = []
    for k in range(5):
        pred += [ full_outputs_list[k][i][0][0] ]
    pred = np.mean(pred, 0)
    ans = (pred > 0.425).astype(np.uint8) * 255
    ans = np.array([ans, ans, ans])
    if ans.sum() == 0:
        hui
    ans = np.transpose(ans, (1, 2, 0))
    f = f.split('/')[-1]
    cv2.imwrite(f'ss/{f}', ans)

!zip -r -j drive/MyDrive/ss_final.zip ss/*